## Question
What impact does a country's economic status (as measured by GDP and other factors) affect the number of human trafficking cases reported in that country?

### Model
Based on a known dataset, can multiple regression be used to determine any relationship between the number of victims of human trafficking each year in a given country and that country's poverty rate, GDP and population?

### Output
We hope to find a line where as GDP, unemployment and other factors change, the number of human trafficking cases may be predicted. If the number of cases can be predicted/modeled, one of two outcomes may be possible:
* Examine existing cases for possible under-reporting and encouraging law enforcement to act more pro-actively.
* Allow leadership in given countries the opportunity to curb trafficking by improving economic conditions within their respective country.

In [24]:
# Import Dependencies
import os
import pandas as pd
import numpy as np

from sqlalchemy import create_engine
from config import conn

# from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LinearRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
import matplotlib.pyplot as plt

In [25]:
# Get trafficking data
trafficking_df = pd.read_sql_table("Trafficking_Cleaned", conn)
trafficking_df

,yearOfRegistration,Datasource,gender,citizenship,isForcedLabour,ControlCategory,RecruiterCategory,ExploitType,Labor_Type,ageCategories
0,2012,Case Management,Female,LK,1,Threats,Other,Forced Labor,Domestic Work,Age 30-38
1,2012,Case Management,Female,LK,1,Financial,Other,Forced Labor,Domestic Work,Age 30-38
2,2012,Case Management,Female,LK,1,Threats,Other,Forced Labor,Domestic Work,Age 30-38
3,2012,Case Management,Female,LK,1,Financial,Other,Forced Labor,Domestic Work,Age 30-38
4,2012,Case Management,Female,LK,1,Financial,Other,Forced Labor,Domestic Work,Age 30-38
...,...,...,...,...,...,...,...,...,...,...
14294,2018,Hotline,Male,US,0,Threats,Family/Relative,Sexual Exploitation,Unknown,Age 9-17
14295,2018,Hotline,Male,US,0,Threats,Family/Relative,Sexual Exploitation,Unknown,Age 9-17
14296,2018,Hotline,Male,US,0,Threats,Family/Relative,Sexual Exploitation,Unknown,Age 9-17
14297,2018,Hotline,Male,US,0,Other,Family/Relative,Sexual Exploitation,Unknown,Age 9-17


In [26]:
# Get total case counts for each country by year
cases_by_year = trafficking_df.groupby(["citizenship", "yearOfRegistration"]).size().reset_index(name="case_counts")
cases_by_year

,citizenship,yearOfRegistration,case_counts
0,0,2015,811
1,0,2016,1860
2,0,2017,2448
3,0,2018,3234
4,AF,2013,3
5,AF,2014,1
6,BY,2015,90
7,BY,2016,112
8,BY,2017,30
9,CN,2017,11


In [27]:
# Get country data
countries_df = pd.read_sql_table("Country_Codes", conn)
countries_df

,country_name,country_iso2,country_iso3,country_un_code
0,Afghanistan,AF,AFG,4
1,Aland Islands,AX,ALA,248
2,Albania,AL,ALB,8
3,Algeria,DZ,DZA,12
4,American Samoa,AS,ASM,16
...,...,...,...,...
242,Wallis and Futuna Islands,WF,WLF,876
243,Western Sahara,EH,ESH,732
244,Yemen,YE,YEM,887
245,Zambia,ZM,ZMB,894


In [28]:
# Merge our country code cross-reference to our aggregated cases per country per year
trafficking_x_countries = cases_by_year.merge(countries_df, how='left', left_on='citizenship', right_on='country_iso2')
trafficking_x_countries

,citizenship,yearOfRegistration,case_counts,country_name,country_iso2,country_iso3,country_un_code
0,0,2015,811,NaN,NaN,NaN,NaN
1,0,2016,1860,NaN,NaN,NaN,NaN
2,0,2017,2448,NaN,NaN,NaN,NaN
3,0,2018,3234,NaN,NaN,NaN,NaN
4,AF,2013,3,Afghanistan,AF,AFG,4
5,AF,2014,1,Afghanistan,AF,AFG,4
6,BY,2015,90,Belarus,BY,BLR,112
7,BY,2016,112,Belarus,BY,BLR,112
8,BY,2017,30,Belarus,BY,BLR,112
9,CN,2017,11,China,CN,CHN,156


In [29]:
# Replace the `0` placeholder for unknowns with null/NaN values.
trafficking_x_countries["citizenship"].replace({'0': np.nan}, inplace=True)
trafficking_x_countries.dropna(inplace=True)
trafficking_x_countries

,citizenship,yearOfRegistration,case_counts,country_name,country_iso2,country_iso3,country_un_code
4,AF,2013,3,Afghanistan,AF,AFG,4
5,AF,2014,1,Afghanistan,AF,AFG,4
6,BY,2015,90,Belarus,BY,BLR,112
7,BY,2016,112,Belarus,BY,BLR,112
8,BY,2017,30,Belarus,BY,BLR,112
9,CN,2017,11,China,CN,CHN,156
10,CN,2018,12,China,CN,CHN,156
11,ER,2014,1,Eritrea,ER,ERI,232
12,HT,2012,89,Haiti,HT,HTI,332
13,ID,2014,1,Indonesia,ID,IDN,360


In [30]:
# Get the population data from our database, convert to DataFrame and merge on to our aggregated data.
# poverty_df = pd.read_sql_table("WorldBank_Poverty", conn, index_col=["ISO"])
# our poverty dataset got saved incorrectly somewhere along the way, will need to fix this bug
gdp_df = pd.read_sql_table("WEO_GDP", conn, index_col=["ISO"])
popn_df = pd.read_sql_table("WEO_Population", conn, index_col=["ISO"])

# year_poverty = []
year_gdp = []
year_popn = []

for row in trafficking_x_countries.iterrows():
    # year_poverty.append(poverty_df.loc[row[1]['country_iso3'], str(row[1]['yearOfRegistration'])])
    year_gdp.append(gdp_df.loc[row[1]['country_iso3'], str(row[1]['yearOfRegistration'])])
    year_popn.append(popn_df.loc[row[1]['country_iso3'], str(row[1]['yearOfRegistration'])])

# Add our results to our cumulative DataFrame
# trafficking_x_countries['Poverty'] = year_poverty
trafficking_x_countries['GDP'] = year_gdp
trafficking_x_countries['Popn'] = year_popn
trafficking_x_countries

,citizenship,yearOfRegistration,case_counts,country_name,country_iso2,country_iso3,country_un_code,GDP,Popn
4,AF,2013,3,Afghanistan,AF,AFG,4,754.402,26.736
5,AF,2014,1,Afghanistan,AF,AFG,4,747.622,27.601
6,BY,2015,90,Belarus,BY,BLR,112,5941.240,9.481
7,BY,2016,112,Belarus,BY,BLR,112,5022.470,9.498
8,BY,2017,30,Belarus,BY,BLR,112,5757.290,9.505
9,CN,2017,11,China,CN,CHN,156,8823.460,1390.080
10,CN,2018,12,China,CN,CHN,156,9919.810,1395.380
11,ER,2014,1,Eritrea,ER,ERI,232,786.540,3.311
12,HT,2012,89,Haiti,HT,HTI,332,1337.340,10.251
13,ID,2014,1,Indonesia,ID,IDN,360,3533.610,252.165


In [31]:
# Create our X and y sets
y = trafficking_x_countries["case_counts"]
X = trafficking_x_countries[["GDP", "Popn"]]

In [32]:
# Create our training and testing splits
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=52)

In [33]:
# Create and train our multiple regression model
multi_model = LinearRegression()
multi_model.fit(X_train, y_train)

LinearRegression()

In [34]:
predict_cases = multi_model.predict(X_test)
predict_cases

array([111.33798913,  58.20170688,  45.07999265,  25.40013563,
        51.42766785, 915.46254032, 500.0347427 ,  44.58053017,
        39.78371021,  39.99702652,  44.69554848, 167.79813691])

In [35]:
y_test

7     112
13      1
29      3
10     12
12     89
45    631
25     12
28     20
5       1
4       3
30    102
33     19
Name: case_counts, dtype: int64

In [36]:
predict_single_case = multi_model.predict([[1810.270, 32.121]])
predict_single_case

array([56.494031])